# Lesson 2: RAG Triad of metrics

In [29]:
import warnings
warnings.filterwarnings('ignore')

In [30]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

In [31]:
from trulens_eval import Tru # setup true object 

tru = Tru()
tru.reset_database() # reset database => recored prompt response 

In [32]:
from llama_index import SimpleDirectoryReader # read pdf document

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [33]:
from llama_index import Document

document = Document(text="\n\n".\
                    join([doc.text for doc in documents]))
#merge all document to one 

In [34]:
from utils import build_sentence_window_index

from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

sentence_index = build_sentence_window_index(
    document,
    llm, # for competion
    embed_model="local:BAAI/bge-small-en-v1.5", # embedding model 
    save_dir="sentence_index" # index with sentence index
)

In [35]:
from utils import get_sentence_window_query_engine

sentence_window_engine = \
get_sentence_window_query_engine(sentence_index)

In [36]:
output = sentence_window_engine.query(
    "What most importance quality for AI carrer")
output.response

'The most important quality for a career in AI is having a strong network and community. This can be achieved by attending meetups and events such as Pie & AI, where you can build connections with like-minded individuals. Additionally, being polite, professional, and grateful to the people you interview with is crucial. It is also important to pay it forward and help others in the DeepLearning.AI community by offering informational interviews. Strengthening your math background is also essential, as it enables you to understand and debug algorithms. However, the depth of mathematical knowledge required may vary over time.'

## Feedback functions

In [37]:
import nest_asyncio

nest_asyncio.apply()

In [38]:
from trulens_eval import OpenAI as fOpenAI

provider = fOpenAI()

### 1. Answer Relevance

In [39]:
from trulens_eval import Feedback

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


### 2. Context Relevance

In [40]:
from trulens_eval import TruLlama

context_selection = TruLlama.select_source_nodes().node.text

In [41]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [42]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


### 3. Groundedness

In [43]:
from trulens_eval.feedback import Groundedness

grounded = Groundedness(groundedness_provider=provider)

In [44]:
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons,
             name="Groundedness"
            )
    .on(context_selection)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


## Evaluation of the RAG application

In [45]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode

tru_recorder = TruLlama(
    sentence_window_engine,
    app_id="App_1",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

In [46]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [47]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?']

In [48]:
eval_questions.append("How can I be successful in AI?")

In [49]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'How can I be successful in AI?']

In [50]:
for question in eval_questions:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

In [51]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

app_id  \
0  App_1   
1  App_1   
2  App_1   
3  App_1   
4  App_1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [52]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,Groundedness,Answer Relevance,Context Relevance
0,"""What are the keys to building a career in AI?""","""The keys to building a career in AI are learning foundational technical skills, working on projects, and finding a job, all of which is supported by being part of a community.""",1.000,1.0,0.40
1,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by allowing individuals to leverage the expertise and insights of their colleagues. When working on larger AI projects that require collaboration, the ability to lead and work effectively as a team becomes crucial. By working together, team members can apply their deep technical insights, make informed decisions, and improve the project as a whole. Additionally, being surrounded by colleagues who are dedicated, hardworking, and continuously learning can inspire individuals to do the same, leading to greater success in AI endeavors.""",0.950,1.0,0.75
2,"""What is the importance of networking in AI?""","""Networking is important in AI because it allows individuals to connect with others who have experience and knowledge in the field. By reaching out to people in their network, such as friends or alumni, individuals can receive guidance and support as they navigate their career in AI. Additionally, attending meetups and informational interviews can help individuals expand their network and learn from professionals who hold positions aligned with their interests. This networking can provide valuable insights, help individuals stay updated on the evolving field of AI, and potentially open up opportunities for future collaborations or job prospects.""",1.000,1.0,0.00
3,"""What are some good habits to develop for a successful career?""","""Developing good habits in areas such as eating, exercise, sleep, personal relationships, work, learning, and self-care can contribute to a successful career. These habits can help individuals move forward while staying healthy and ultimately achieve better outcomes for themselves.""",1.000,1.0,0.30
4,"""How can altruism be beneficial in building a career?""","""Altruism can be beneficial in building a career by helping others even as one focuses on their own career growth. By aiming to lift others during every step of their own journey, individuals can achieve better outcomes for themselves. This can create a positive reputation and build strong relationships, which can lead to opportunities and support in their career advancement. Additionally, practicing altruism can enhance personal satisfaction and fulfillment, which can contribute to overall well-being and success in one's career.""",0.375,1.0,0.45
5,"""What is imposter syndrome and how does it relate to AI?""","""Imposter syndrome is a psychological phenomenon where individuals doubt their abilities and fear being exposed as frauds, despite evidence of their success. In the context of AI, newcomers to the field sometimes experience imposter syndrome, questioning whether they truly belong in the AI community and if they are capable of contributing to the field. It is important to address imposter syndrome in order to encourage individuals to continue growing in AI and not let self-doubt hinder their progress.""",1.000,0.9,0.40
6,"""Who are some accomplished individuals who have experienced imposter syndrome?""","""Sheryl Sandberg, Michelle Obama, Tom Hanks, and Mike Cannon-Brookes are some accomplished individuals who have experienced imposter syndrome.""",1.000,1.0,0.45
7,"""What is the first step to becoming good at AI?""","""The first step to becoming good at AI is to learn foundational technical skills.""",1.000,1.0,0.40
8,"""What are some common challenges in AI?""","""Some common challenges in AI include keeping up-to-date with evolving technology, finding suitable projects and estimating timelines and return on investment, managing the highly iterative nature of AI projects, collaborating with stakeholders who lack expertise in AI, a

In [53]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
App_1,0.8975,0.99,0.47,6.272727,0.000881


In [54]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-31-7-241p57896.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>